In [2]:
import pandas as pd
from src.data.hex_utils import * 
import pickle
import numpy as np

In [3]:
with open("../data/processed/geolife_hex_50.pkl", "rb") as f:
    hdf = pickle.load(f)

In [ ]:
hdf

In [ ]:
hdf

In [5]:
hdf

,q,r,datetime,user,t_idx,cell_dist,is_workday,is_in_time_0,is_in_time_1,is_in_time_2,is_in_time_3
312042,61,80,2008-10-23 02:53:04.000000000,0,1,1.0,True,True,False,False,False
312042,21,53,2008-10-23 02:53:04.000000000,0,1,1.0,True,True,False,False,False
312054,20,54,2008-10-23 02:54:05.000000000,0,1,1.0,True,True,False,False,False
312109,19,54,2008-10-23 02:58:40.000000000,0,1,1.0,True,True,False,False,False
0,18,55,2008-10-23 03:36:41.000000000,0,1,1.0,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
17,31,35,2007-12-08 05:55:15.076923076,181,17118,1.0,False,True,False,False,False
12420255,32,34,2007-12-08 05:58:54.000000000,181,17118,1.0,False,True,False,False,False
12420265,32,35,2007-12-08 06:10:43.000000000,181,17118,1.0,False,True,False,False,False
12420277,32,34,2007-12-08 07:03:03.000000000,181,17118,1.0,False,False,True,False,False


In [16]:
actions = {
    (0, 1): 0,
    (0, -1): 1,
    (1, 0): 2,
    (-1, 0): 3,
    (1, -1): 4,
    (-1, 1): 5
}

In [11]:
tmp = []
for t_idx, tdf in hdf.groupby("t_idx"):
    tdf_shift = tdf.shift(-1)
    tdf["delta_q"] = tdf_shift["q"] - tdf["q"]
    tdf["delta_r"] = tdf_shift["r"] - tdf["r"]
    tmp.append(tdf)
ndf = pd.concat(tmp)

In [14]:
ndf['delta_q'] = -ndf['delta_q']
ndf['delta_r'] = -ndf['delta_r']

In [18]:
ndf['moves'] = ndf.apply(lambda row: actions.get((row['delta_q'], row['delta_r']), -1), axis=1)

In [12]:
ndf = ndf[ndf['q'] <= 60]

In [19]:
ndf

,q,r,datetime,user,t_idx,cell_dist,is_workday,is_in_time_0,is_in_time_1,is_in_time_2,is_in_time_3,delta_q,delta_r,moves
312042,21,53,2008-10-23 02:53:04.000000000,0,1,1.0,True,True,False,False,False,-1.0,1.0,5
312054,20,54,2008-10-23 02:54:05.000000000,0,1,1.0,True,True,False,False,False,-1.0,-0.0,3
312109,19,54,2008-10-23 02:58:40.000000000,0,1,1.0,True,True,False,False,False,-1.0,1.0,5
0,18,55,2008-10-23 03:36:41.000000000,0,1,1.0,True,True,False,False,False,-1.0,1.0,5
312190,17,56,2008-10-23 04:08:07.000000000,0,1,1.0,True,True,False,False,False,-0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,31,36,2007-12-08 05:51:36.153846153,181,17118,1.0,False,True,False,False,False,-0.0,-1.0,1
17,31,35,2007-12-08 05:55:15.076923076,181,17118,1.0,False,True,False,False,False,1.0,-1.0,4
12420255,32,34,2007-12-08 05:58:54.000000000,181,17118,1.0,False,True,False,False,False,-0.0,1.0,0
12420265,32,35,2007-12-08 06:10:43.000000000,181,17118,1.0,False,True,False,False,False,-0.0,-1.0,1
